In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim
from scipy.stats import unitary_group
from numpy import array
import matplotlib.pyplot as plt

In [3]:
def raising_operator(dim):
    matrix = np.zeros([dim,dim]) *1j
    for i in range(dim-1):
        matrix[i+1][i] = np.sqrt(i+1)
    return matrix

def lowering_operator(dim):
    matrix = np.zeros([dim,dim])* 1j
    for i in range(dim-1):
        matrix[i][i+1] = np.sqrt(i+1)
    return matrix

def Hstatic(dim_c, dim_q, w_t, w_c, K, A, x, X):
    ar = raising_operator(dim_c)
    al = lowering_operator(dim_c)
    br = raising_operator(dim_q)
    bl = lowering_operator(dim_q)
    
    H_o = w_c*np.kron(ar@al,np.identity(dim_q)) + K/2*np.kron(ar@ar@al@al,np.identity(dim_q))
    H_t = w_t*np.kron(np.identity(dim_c),br@bl) + A/2*np.kron(np.identity(dim_c),br@br@bl@bl)
    H_i = x*np.kron(ar@al,br@bl)+X/2*np.kron(ar@ar@al@al,br@bl,)
    Hstatic = H_o + H_t + H_i
    return Hstatic

def cQED_hamiltonian_terms(dim_c, dim_q, w_t, w_c, K, A, x, X):
    ar = raising_operator(dim_c)
    al = lowering_operator(dim_c)
    br = raising_operator(dim_q)
    bl = lowering_operator(dim_q)
    
    h_1  = np.kron(al, np.identity(dim_q))+np.kron(ar, np.identity(dim_q))
    h_2 = np.kron(np.identity(dim_c),bl)+np.kron(np.identity(dim_c),br)
    h_1c  = ((np.kron(al, np.identity(dim_q)))-np.kron(ar, np.identity(dim_q)))*1j
    h_2c = ((np.kron(np.identity(dim_c),bl))-np.kron(np.identity(dim_c),br))*1j 
    h_d = Hstatic(dim_c, dim_q, w_t, w_c, K, A, x, X)
    return h_1, h_2, h_1c, h_2c, h_d


def hamiltonian(w1, w1_c, w2, w2_c):
    # Omega_c: cavity frequency
    # Omega_a: atom frequency
    # g: coupling strength
    H_1 = w1 * torch.tensor(h_1*10**6, dtype=torch.complex128,)
    H_1c = w1_c * torch.tensor(h_1c*10**6 , dtype=torch.complex128)
    H_2 = w2 * torch.tensor(h_2*10**6, dtype=torch.complex128)
    H_2c = w2_c * torch.tensor(h_2c*10**6, dtype=torch.complex128)

    H = H_1 + H_1c + H_2 + H_2c + torch.tensor(h_d)
    return H

def projector(dim_c, dim_b):
    p = torch.zeros([dim_c*2, dim_c*2],dtype=torch.complex128,)
    for i in range (2*dim_b, 2*dim_c):
        p[i][i] = dim_c/(dim_c-dim_b) # normalize to match the fid computer 
    return p
import quimb as qu
import quimb.tensor as qtn
def noisy_tensors(n_bond_lvl,L=1,error_rate =[0,0,0]):
    
    # we approximate Linbladian evolution as a unitary evolution followed by the collapse operators
    # let's define the collapse operators as a MPO
    
    tlist = []
    p_rel =  lowering_operator(2)
    p_dep =  raising_operator(2) @ lowering_operator(2)
    b_rel = lowering_operator(n_bond_lvl)
    
    p_rel_tensor = np.zeros([4,2,2])+0j
    p_rel_tensor[0] = np.eye(2)
    p_rel_tensor[1] = error_rate[0] *p_rel
    p_rel_tensor[2] = error_rate[0] * p_rel@np.conj(p_rel.T) 
    p_rel_tensor[3] = error_rate[0] *np.eye(2)
    
    pc_rel_tensor = np.zeros([4,2,2])+0j
    pc_rel_tensor[0] = np.eye(2)
    pc_rel_tensor[1] = np.conj(p_rel)
    pc_rel_tensor[2] = -1/2*np.eye(2)
    pc_rel_tensor[3] = -1/2*(p_rel@ np.conj(p_rel.T) )
    
    p_dep_tensor = np.zeros([4,2,2])+0j
    p_dep_tensor[0] = np.eye(2)
    p_dep_tensor[1] = error_rate[1] *p_dep
    p_dep_tensor[2] = error_rate[1] * p_dep@np.conj(p_dep.T) 
    p_dep_tensor[3] = error_rate[1] *np.eye(2)
    
    pc_dep_tensor = np.zeros([4,2,2])+0j
    pc_dep_tensor[0] = np.eye(2)
    pc_dep_tensor[1] = np.conj(p_dep)
    pc_dep_tensor[2] = -1/2*np.eye(2)
    pc_dep_tensor[3] = -1/2*(p_dep@ np.conj(p_dep.T) )
    
    b_rel_tensor = np.zeros([4,n_bond_lvl,n_bond_lvl])+0j
    b_rel_tensor[0] = np.eye(n_bond_lvl)
    b_rel_tensor[1] = error_rate[2] *b_rel
    b_rel_tensor[2] = error_rate[2] * b_rel@np.conj(b_rel.T) 
    b_rel_tensor[3] = error_rate[2] *np.eye(n_bond_lvl)
    
    bc_rel_tensor = np.zeros([4,n_bond_lvl,n_bond_lvl])+0j
    bc_rel_tensor[0] = np.eye(n_bond_lvl)
    bc_rel_tensor[1] = np.conj(b_rel)
    bc_rel_tensor[2] = -1/2*np.eye(n_bond_lvl)
    bc_rel_tensor[3] = -1/2*(b_rel@ np.conj(b_rel.T) )
    
    for i in range (L):
        inds_d = f'a_rel{i}',f'p_rel{i}',f'p_dep{i}'
        t = qtn.Tensor(p_rel_tensor,inds_d,tags=f'site{i}')
        inds_d = f'a_rel{i}',f'pc_rel{i}',f'pc_dep{i}'
        tc = qtn.Tensor(pc_rel_tensor,inds_d,tags=f'site{i}')
        tlist.append(t&tc)
        
        inds_d =f'a_dep{i}',f'p_dep{i}',f'p_out{i}'
        t = qtn.Tensor(p_dep_tensor,inds_d,tags=f'site{i}')
        inds_d =f'a_dep{i}',f'pc_dep{i}',f'pc_out{i}'
        tc = qtn.Tensor(pc_dep_tensor,inds_d,tags=f'site{i}')
        tlist.append(t&tc)
        # add in error models for cavity
        inds_d =f'b{i}',f'b_rel{i}',f'b_out{i}'
        t = qtn.Tensor(b_rel_tensor,inds_d,tags=f'site{i}')
        inds_d =f'b{i}',f'bc_rel{i}',f'bc_out{i}'
        tc = qtn.Tensor(bc_rel_tensor,inds_d,tags=f'site{i}')
        tlist.append(t&tc)

    #tn = tlist[0]
    #for i in range (1,len(tlist)):
    #    tn = tn&tlist[i]
    return ((tlist[0]&tlist[1])^all).data, (tlist[2]^all).data

def objective_function_unitary_sythesis(parameters, dt, n_steps, U_target):
    w1, w1_c, w2, w2_c = parameters
    
    U = torch.eye(dim_c*dim_q, dtype=torch.complex128, requires_grad=True)  # Initialize the unitary
    for i in range(n_steps):
        H = hamiltonian(w1[i], w1_c[i], w2[i], w2_c[i])
        U = U@torch.matrix_exp(-1j * H * dt)
    #print(U)
    
    fidelity = torch.abs(torch.trace(torch.matmul(U_target, U))) / (dim_c*dim_q)
    loss = 1 - fidelity
    
    return loss

def objective_function(parameters, dt, n_steps,noisy):
    w1, w1_c, w2, w2_c = parameters
    
    U = torch.eye(dim_c*dim_q, dtype=torch.complex128, requires_grad=True)  # Initialize the unitary
    for i in range(n_steps):
        H = hamiltonian(w1[i], w1_c[i], w2[i], w2_c[i])
        U = U@torch.matrix_exp(-1j * H * dt)
    #print(U)
    ##loss = 1 - fidelity
    
    penalty = 1 - torch.abs(torch.trace(torch.matmul(projector(dim_c, dim_b), U))) / (dim_c*dim_q)
    mpo = create_mpo(L, ham, burn_in)
    
    
    if dim_c != dim_b:
        #loss = evaluate_mps_mpo_noisy(U, mpo , L, burn_in, noisy) + .2*penalty
        loss = evaluate_mps_mpo_noisy(U, mpo , L, burn_in,noisy) + .2 * penalty
    else:
        loss = evaluate_mps_mpo_noisy(U, mpo , L, burn_in,noisy)
    
    return loss

# Function to contract MPS and MPO tensors and compute the expectation value
from hamiltonian import model_mpo

L, burn_in = 10,5
J, V, h = 1, .5, -1
ham = torch.tensor(model_mpo.sd_ising(J, V, h, L - burn_in),dtype=torch.complex128)

def create_mpo(L, ham, burn_in = 0):
    mpo = []
    for _ in range (0, burn_in):
        mpo.append(torch.eye(10,dtype=torch.complex128).reshape(2,5,2,5))
    for _ in range (burn_in, L):
        mpo.append(ham)
    return mpo

mpo = create_mpo(L, ham, burn_in)

def evaluate_mps_mpo_noisy(unitary, mpo , L, burn_in = 0,noise_tensors = None):
    
    # assume this is a translational invariant MPS
    isometry = unitary.reshape(dim_c,2,dim_c,2) # dim_c,dim_q
    
    # define boundary condition 
    bdry_mps = torch.eye(dim_c,dtype=torch.complex128)[0] 
    bdry_mpo_l = torch.eye(5,dtype = torch.complex128)[-1]#shape (2,5,2,5)
    bdry_mpo_r = torch.eye(5,dtype = torch.complex128)[0]
    
    # Contract input state with the first isometry; ckr: p_out, pc_out,
    if noise_tensors == None or burn_in !=0 :
        mps_state = torch.einsum("a, abcd, i, ilkj, o, jrdo, bl-> ckr", bdry_mps, isometry, bdry_mps, torch.conj(isometry), bdry_mpo_l, mpo[0],torch.tensor([[1,0],[0,0]],dtype = torch.complex128))

    else:
        #p_rel0', 'pc_rel0', 'p_out0', 'pc_out0'; 'b_rel0', 'b_out0', 'bc_rel0', 'bc_out0'
        p_noise, b_noise = noise_tensors
        mps_state = torch.einsum("a, abcd, i, ilkj, o, jrdo, vw, blvw -> ckr", bdry_mps, isometry, bdry_mps, torch.conj(isometry), bdry_mpo_l, mpo[0],torch.tensor([[1,0],[0,0]],dtype = torch.complex128),torch.tensor(p_noise,dtype = torch.complex128))
        mps_state = torch.einsum("ckr,cvkw -> vwr", mps_state, torch.tensor(b_noise,dtype = torch.complex128))

    # Contract the MPS state with the MPO and the remaining isometries
    for i in range(1, L):
        if noise_tensors == None or i < burn_in:
            mps_state = torch.einsum("aip, abcd, ilkj, jrdp, bl-> ckr", mps_state,  isometry, torch.conj(isometry), mpo[i],torch.tensor([[1,0],[0,0]],dtype = torch.complex128))
        else:
            #p_rel0', 'pc_rel0', 'p_out0', 'pc_out0'; 'b_rel0', 'b_out0', 'bc_rel0', 'bc_out0'
            p_noise, b_noise = noise_tensors
            mps_state = torch.einsum("aip, abcd, ilkj, jrdp, vw, blvw-> ckr", mps_state,  isometry, torch.conj(isometry), mpo[i],torch.tensor([[1,0],[0,0]],dtype = torch.complex128),torch.tensor(p_noise,dtype = torch.complex128))
            mps_state = torch.einsum("ckr,cvkw -> vwr", mps_state, torch.tensor(b_noise,dtype = torch.complex128))

    # Contract the final MPS tensor with its conjugate
    result = torch.real(torch.einsum("aap, p-> ", mps_state,bdry_mpo_r))
    return result

# Create the MPO observable (identity)
mpo = create_mpo(6, ham, 0)

# Evaluate the MPS with the MPO observable

dim_b = 4 # bond dimension = useful cavity levels
dim_c = dim_b+6 # of cavity/oscillator levels
dim_q = 2 # of qubit levels; which is usually set to 2

#System Parameter; taken from Yale's
w_c = 0#2*np.pi*4452*10**6
w_t = 0#2*np.pi*5664*10**6
K = -2*np.pi*3.7*1000
A = -2*np.pi*236*10**6
x = -2*np.pi*2.139 *10**6#chi
X = -2*np.pi*19*1000

h_1, h_2, h_1c, h_2c, h_d = cQED_hamiltonian_terms(dim_c, dim_q, w_t, w_c, K, A, x, X)

# Time step and number of steps
dt = 10 * 10**(-9)
n_steps = 100

# Initial parameters
w1 = Variable(torch.tensor(np.random.random(n_steps), dtype=torch.float64), requires_grad=True)
w1_c = Variable(torch.tensor(np.random.random(n_steps) , dtype=torch.float64), requires_grad=True)
w2 = Variable(torch.tensor(np.random.random(n_steps), dtype=torch.float64), requires_grad=True)
w2_c = Variable(torch.tensor(np.random.random(n_steps) , dtype=torch.float64), requires_grad=True)

# Set up the optimizer
optimizer = optim.Adam([w1, w1_c, w2, w2_c], lr=0.2)

noisy = noisy_tensors(dim_c,1,[1/170,1/43,1/2700]) #set up the noise strength

n_epochs = 2000
import time
t0 = time.time()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = objective_function([w1, w1_c, w2, w2_c], dt, n_steps,noisy)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        for param in [w1, w1_c, w2, w2_c]:
            param.clamp_(-10, 10)

    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Loss:{loss}, Time:{time.time()-t0}')

Epoch: 0, Loss:-0.03201468720387668, Time:0.10565400123596191
Epoch: 100, Loss:-3.6785096406708204, Time:9.643810749053955
Epoch: 200, Loss:-3.685018382711881, Time:19.397751092910767
Epoch: 300, Loss:-3.6876212959540613, Time:29.797627925872803
Epoch: 400, Loss:-3.6945061055881068, Time:39.58246302604675
Epoch: 500, Loss:-3.6959788334162886, Time:49.80606269836426
Epoch: 600, Loss:-3.7094379232165844, Time:59.877628803253174
Epoch: 700, Loss:-3.7101195159063725, Time:69.29267191886902
Epoch: 800, Loss:-3.7048019622380104, Time:78.92608094215393
Epoch: 900, Loss:-3.7759172602875513, Time:88.75159192085266
Epoch: 1000, Loss:-3.802116858807922, Time:98.60882782936096
Epoch: 1100, Loss:-3.9101177780933494, Time:109.04788303375244
Epoch: 1200, Loss:-3.9528694364661443, Time:119.50072383880615
Epoch: 1300, Loss:-3.978961392546126, Time:129.33133792877197
Epoch: 1400, Loss:-3.9992598093006135, Time:139.09415888786316
Epoch: 1500, Loss:-4.013397825083274, Time:148.8583607673645
Epoch: 1600, L